### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import accuracy_score
import sys
from keras import regularizers
import pandas

Using TensorFlow backend.


### Load Raw Training Data

In [2]:
inpData = []
labels = []
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
        

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
index = 0
while index + 1 < len(inpData):
    if inpData[index + 1] > maxThreshold:# Scale Up
        target.append(1)
    elif inpData[index + 1] < minThreshold: # Scale Down
        target.append(-1)
    else:
        target.append(0)  #Remain same
    index += 1

In [ ]:
print(target)

### Preprocessing Input Data

In [4]:
cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1
    

In [5]:
cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

In [6]:
#len(inpData)
#X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.01, random_state=42)
newTrainX = cpuUsageData
newTrainY = cpuScalingCategorical
for i in range(1,len(cpuUsageData)):
    scalingVal = np.argmax(cpuScalingCategorical[i])
    if scalingVal == 0:
        for j in range(0,7):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
        
    if scalingVal == 1 :
        for j in range(0,10):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)


X_train, X_test, y_train, y_test = train_test_split(newTrainX , newTrainY, test_size=0.10, random_state=21)

In [ ]:
cpuUsageData.shape
#np.shape(cpuUsage)

In [ ]:
import pandas
testrec = 5
print(cpuScalingCategorical[testrec])
print(cpuScaling[testrec])
print(set(cpuScaling))
myseries=pandas.Series(cpuScaling)
print(myseries.value_counts())

In [7]:
usageInputLayer = keras.layers.Input(shape=(historyParam,1))
flattened_layer = keras.layers.Flatten()(usageInputLayer)
fullyConnected_0 = keras.layers.Dense(125, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(flattened_layer)
fullyConnected_1 = keras.layers.Dense(100, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_0)
#drop1 = keras.layers.Dropout(0.25)(fullyConnected_1)
fullyConnected_2 = keras.layers.Dense(75, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_1)
#drop2 = keras.layers.Dropout(0.25)(fullyConnected_2)
fullyConnected_3 = keras.layers.Dense(50, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_2)
fullyConnected_4 = keras.layers.Dense(25, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_3)
fullyConnected_5 = keras.layers.Dense(12, activation = 'relu',
                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_4)
fullyConnected_6 = keras.layers.Dense(6, activation = 'relu',
                kernel_initializer='he_normal')(fullyConnected_5)
softmax_output = keras.layers.Dense(3,activation='softmax')(fullyConnected_5)
predictionModel = keras.models.Model(inputs=usageInputLayer,outputs=softmax_output)
predictionModel.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 1)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               6375      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12600     
_________________________________________________________________
dense_3 (Dense)              (None, 75)                7575      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                3800      
_________________________________________________________________
dens

In [8]:
def customLoss(rewardInputLayer):
    def loss(y_true,y_pred):
        tmp_pred = keras.layers.Lambda(lambda x: keras.backend.clip(x,0.05,0.95))(y_pred)
        tmp_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
        policy_loss=keras.layers.Multiply()([tmp_loss,rewardInputLayer])
        #policy_loss = tf.reduce_sum(rewardInputLayer,axis =-1)
        return policy_loss
    return loss

In [9]:
rewardInputLayer = keras.layers.Input(shape=(1,),name='rewardInputLayer')
TrainingModel = keras.models.Model(inputs=[usageInputLayer,rewardInputLayer],outputs=softmax_output)
TrainingModel.compile(optimizer="adam",loss=customLoss(rewardInputLayer),metrics=['accuracy'])

In [10]:
TrainingModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 1)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               6375      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12600     
_________________________________________________________________
dense_3 (Dense)              (None, 75)                7575      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
__________

In [11]:
def generateReward(usage,action,minThreshold,maxThreshold):
    if usage[49] > maxThreshold :
        if action == 0:
            return 1
        elif action == -1:
            return 2
    elif usage[49] < minThreshold: 
        if action == 0:
            return 1
        elif action == 1:
            return 2
    else:
        if action != 0:
            return 1
    return -1


def generateReward2(requiredAction,action):
    predDict = {0:0,1:1,2:-1}
    actualActions = predDict[np.argmax(requiredAction)]
    if(actualActions == action):
        reward = 1
    else:
        reward = -abs(actualActions-action)
    return reward
    
def simulateSituation(pModel,tModel,usageArray,minThreshold,maxThreshold):
    predDict = {0:0,1:1,2:-1}
    #modelInput = np.expand_dims(usageArray,axis=0)
    res = pModel.predict(usageArray)
    #mapper = lambda x: predDict[x]
    actionPredictedOneHot = to_categorical(np.argmax(res,axis = 1),num_classes = 3)
    actionPredicted = np.array([predDict[np.argmax(x)] for x in res])
    rewardsList = []
    #print(actionPredicted," \n",usageArray[0])
    print(len(usageArray))
    
    for i in range(0,len(usageArray)):
        #actionReward = generateReward(y_train[i],actionPredicted[i],minThreshold,maxThreshold,)
        actionReward = generateReward2(y_train[i],actionPredicted[i])
        if actionReward == None:
            print(actionPredicted[i]," \n",usageArray[i]," ", minThreshold," ",maxThreshold)
            sys.exit()
            
        rewardsList.append(actionReward)
        
    rewardsArray = np.array(rewardsList)
    actionSeries = pandas.Series(actionPredicted)
    print(actionSeries.value_counts())
    rewardSeries = pandas.Series(rewardsList)
    print(rewardSeries.value_counts())
    #print(rewardsArray.shape," ",rewardsArray[0])
    #if os.path.isfile("LSTMRLBestWeights.hdf5"):
    #    print("Loading Previous weights!")
    #    tModel.load_weights("LSTMRLBestWeights.hdf5")
    history = tModel.fit(x = [usageArray,rewardsArray], y = actionPredictedOneHot,epochs = 1)
    #print(usageArray[0]," ", actionPredicted," \n", rewardsArray)
    return pModel,tModel,res
    

In [14]:
#filepath="LSTMRLBestWeights.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

epochs = 50
for i in range(epochs):
    _,_,modelOutputOneHot = simulateSituation(predictionModel,TrainingModel,X_train,30,70)
    #simulateSituation(predictionModel,TrainingModel,X_train,30,70)
    res = predictionModel.predict(X_test)
    print(accuracy_score(np.argmax(modelOutputOneHot,axis = 1),np.argmax(y_train,axis = 1)))
    print(accuracy_score(np.argmax(res,axis = 1),np.argmax(y_test,axis = 1)))

11345
 0    4243
-1    3613
 1    3489
dtype: int64
 1    10241
-1     1012
-2       92
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 105us/step - loss: 0.2839 - acc: 0.9815
0.9026884089907449
0.8691514670896114
11345
 0    4694
-1    3516
 1    3135
dtype: int64
 1    9964
-1    1243
-2     138
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 106us/step - loss: 0.2073 - acc: 0.9148
0.8782723666813574
0.8398096748612213
11345
 1    4233
 0    3908
-1    3204
dtype: int64
 1    9602
-1    1735
-2       8
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 106us/step - loss: -0.2713 - acc: 0.8550
0.8463640370207139
0.8017446471054719
11345
-1    5258
 1    3227
 0    2860
dtype: int64
 1    9084
-1    1941
-2     320
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 105us/step - loss: -1.4727 - acc: 0.7778
0.8007051564565888
0.8699444885011896
11345
 0    4942
 1    3421
-1    2982
dtype: int64

11345/11345 [==============================] - 1s 106us/step - loss: 0.2900 - acc: 0.9862
0.9117672983693257
0.9111816019032514
11345
 0    4323
-1    3599
 1    3423
dtype: int64
 1    10314
-1      958
-2       73
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 105us/step - loss: 0.2669 - acc: 0.9932
0.9091229616571177
0.9056304520222046
11345
 0    4413
-1    3591
 1    3341
dtype: int64
 1    10296
-1      976
-2       73
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 107us/step - loss: 0.2663 - acc: 0.9852
0.9075363596297928
0.9111816019032514
11345
 0    4133
-1    3643
 1    3569
dtype: int64
 1    10286
-1      986
-2       73
dtype: int64
Epoch 1/1
11345/11345 [==============================] - 1s 108us/step - loss: 0.2628 - acc: 0.9766
0.9066549140590568
0.9072164948453608
11345
 0    4308
 1    3521
-1    3516
dtype: int64
 1    10295
-1      983
-2       67
dtype: int64
Epoch 1/1
11345/11345 [==============================]

In [15]:
predictionModel.save("3OutputRL3BestWeights.hdf5")

In [ ]:
if os.path.isfile("3OutputRLBestWeights.hdf5"):
    print("Loading Previous weights!")
    TrainingModel.load_weights("3OutputRLBestWeights.hdf5")
res = predictionModel.predict(X_train)

In [ ]:
accuracy_score(np.argmax(res,axis = 1),np.argmax(y_train,axis = 1))

In [ ]:
print(to_categorical(np.argmax(y_train,axis = 1),num_classes = 3)[:10,:])
print(y_train[:10,:])

In [ ]:
np.argmax([[0,0,1],[1,0,0]],axis = 1)

In [ ]:
temp1 = np.array([[]])

In [ ]:
res = predictionModel.predict(X_train)

In [ ]:
res.shape

In [ ]:
predDict = {0:0,1:1,2:-1}
actionPredictedOneHot = to_categorical(np.argmax(res,axis = 1),num_classes = 3)
actionPredicted = np.array([predDict[np.argmax(x)] for x in res])
rewardList = []
for i in range(0,len(y_train)):
        #actionReward = generateReward(y_train[i],actionPredicted[i],minThreshold,maxThreshold,)
        actionReward = generateReward2(y_train[i],actionPredicted[i])
        rewardList.append(actionReward)
actionSeries = pandas.Series(actionPredicted)
print(actionSeries.value_counts())
rewardSeries = pandas.Series(rewardList)
print(rewardSeries.value_counts())

In [ ]:
resSeries = pandas.Series([predDict[np.argmax(x)] for x in y_train])
print(resSeries.value_counts())

In [ ]:
filepath="LSTMBestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
if os.path.isfile("LSTMBestWeights.hdf5"):
    print("Loading Previous weights!")
    model.load_weights("LSTMBestWeights.hdf5")

In [ ]:
history = model.fit(X_train, y_train, epochs = 15,validation_data = [X_test, y_test], callbacks = callbacks_list,verbose=0)

In [ ]:
X_test.shape
np.expand_dims(X_test[0],axis = 0).shape

In [ ]:
res = model.predict(np.expand_dims(X_test[0],axis=0))

In [ ]:
print(res)
print(np.argmax(res[0]))

In [ ]:
np.argmax([0,1,0],axis = 0)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])

In [ ]:
cpuUsageData
cpuScalingCategorical
newTrainX = cpuUsageData
newTrainY = cpuScalingCategorical
for i in range(1,len(cpuUsageData)):
    scalingVal = np.argmax(cpuScalingCategorical[i])
    if scalingVal == 0:
        for j in range(0,7):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
        
    if scalingVal == 1 :
        for j in range(0,10):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
#newTrainX = [cpuUsageData[x] for x in range(1,len(cpuUsageData)] if np.argmax(cpuScalingCategorical[x])==0 ]

res2Series = pandas.Series([predDict[np.argmax(x)] for x in newTrainY])
print(res2Series.value_counts())